In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
from tcompress import register_ as reg
from tcompress import algo_cooling as algo
from tcompress import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from tqdm import tqdm


In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False)

In [3]:
#ITF params
J = 1
h = 3.05
delta = 0.05

In [4]:
Lx, Ly = 10, 10
L = Lx * Ly
edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)

sites = sorted({ (site,) for edge in edges for site in edge})
N = len(sites)
N

100

In [5]:
mpo=qtn.MPO_identity(N, phys_dim=2, dtype="complex128")
mpo.apply_to_arrays(to_backend_c)


In [6]:
pbar = tqdm(total=len(edges), desc="mpo", ncols=100, dynamic_ncols=True)

chi = 120
for count, site in enumerate(sites):
    phi = torch.tensor( -h * delta )
    rx = qtn.circuit.rx_gate_param_gen([phi])
            
    ((x, y),) = site
    where_ = (y*Lx+x, )
    quf.apply_mpo_swap_bra(mpo, [to_backend_c(rx)], [where_], contract = True,
                        max_bond = None,to_backend=to_backend_c,
                        )


for count, edge in enumerate(edges):

    phi = torch.tensor( -J * delta *2 )
    rzz = qtn.circuit.rzz_param_gen([phi])

    
    ((x0, y0),(x1, y1)) = edge
    where_ = (y0*Lx+x0, y1*Lx+x1)
    quf.apply_mpo_swap_bra(mpo, [to_backend_c(rzz)], [where_], contract = 'split', cutoff = 1.0e-12,
                        max_bond = None,to_backend=to_backend_c,
                        )
    mpo.compress("left", max_bond=chi, **{"cutoff":1.e-12})
    # print(mpo.show())

    pbar.set_postfix({
            "bond": f"{mpo.max_bond()}",
            #"||grad||": f"{grad_norm:.2e}"
        })
    pbar.update(1)

for count, site in enumerate(sites):
    phi = torch.tensor( -h * delta )
    rx = qtn.circuit.rx_gate_param_gen([phi])
            
    ((x, y),) = site
    where_ = (y*Lx+x, )
    quf.apply_mpo_swap_bra(mpo, [to_backend_c(rx)], [where_], contract = True, 
                        tags="G", dtype="complex128", cutoff = 1.0e-12,
                        max_bond = None,to_backend=to_backend_c,
                        )


mpo:   0%|          | 0/180 [00:00<?, ?it/s]/Users/rezah/Documents/time_compressed_algo/algo_cooling.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=dtype, device=device, requires_grad=requires_grad)
mpo:  57%|█████▋    | 103/180 [00:05<00:16,  4.59it/s, bond=57]

_LinAlgError: linalg.svd: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 1).

In [ ]:
mpo.show()